In [73]:
import os
import sys
import numpy as np
from nltk.classify import NaiveBayesClassifier
import nltk
import re

In [74]:
TRAIN_FILE = 'data/ratings_train.txt'
TEST_FILE = 'data/ratings_test.txt'

In [75]:
!head data/ratings_train.txt

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1


In [76]:
!head data/ratings_test.txt

id	document	label
6270596	굳 ㅋ	1
9274899	GDNTOPCLASSINTHECLUB	0
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0
6723715	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??	0
7898805	음악이 주가 된, 최고의 음악영화	1
6315043	진정한 쓰레기	0
6097171	마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다	0
8932678	갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다	0


In [77]:
train_reviews = []
test_reviews = []

In [78]:
# Read data
with open(TRAIN_FILE, 'r') as f:
    next(f) # header skip
    for line in f.readlines():
        _, review, label = line.strip().split('\t')
        label = 'pos' if label == '1' else 'neg'
        train_reviews.append((review.strip(), label.strip()))
        
with open(TEST_FILE, 'r') as f:
    next(f) # header skip
    for line in f.readlines():
        _, review, label = line.strip().split('\t')
        label = 'pos' if label == '1' else 'neg'
        test_reviews.append((review.strip(), label.strip()))

In [79]:
train_reviews[:10]

[('아 더빙.. 진짜 짜증나네요 목소리', 'neg'),
 ('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'pos'),
 ('너무재밓었다그래서보는것을추천한다', 'neg'),
 ('교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'neg'),
 ('사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', 'pos'),
 ('막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', 'neg'),
 ('원작의 긴장감을 제대로 살려내지못했다.', 'neg'),
 ('별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  'neg'),
 ('액션이 없는데도 재미 있는 몇안되는 영화', 'pos'),
 ('왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', 'pos')]

In [80]:
test_reviews[:10]

[('굳 ㅋ', 'pos'),
 ('GDNTOPCLASSINTHECLUB', 'neg'),
 ('뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아', 'neg'),
 ('지루하지는 않은데 완전 막장임... 돈주고 보기에는....', 'neg'),
 ('3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??', 'neg'),
 ('음악이 주가 된, 최고의 음악영화', 'pos'),
 ('진정한 쓰레기', 'neg'),
 ('마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다', 'neg'),
 ('갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다',
  'neg'),
 ('이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..', 'pos')]

In [81]:
def tokenizer(sentence):
    tokens = re.findall(r"[\w]+|[^\s\w]", sentence)
    return tokens

In [82]:
train_reviews = [(tokenizer(review), label) for review, label in train_reviews]
test_reviews = [(tokenizer(review), label) for review, label in test_reviews]

In [83]:
train_reviews[:5]

[(['아', '더빙', '.', '.', '진짜', '짜증나네요', '목소리'], 'neg'),
 (['흠',
   '.',
   '.',
   '.',
   '포스터보고',
   '초딩영화줄',
   '.',
   '.',
   '.',
   '.',
   '오버연기조차',
   '가볍지',
   '않구나'],
  'pos'),
 (['너무재밓었다그래서보는것을추천한다'], 'neg'),
 (['교도소', '이야기구먼', '.', '.', '솔직히', '재미는', '없다', '.', '.', '평점', '조정'], 'neg'),
 (['사이몬페그의',
   '익살스런',
   '연기가',
   '돋보였던',
   '영화',
   '!',
   '스파이더맨에서',
   '늙어보이기만',
   '했던',
   '커스틴',
   '던스트가',
   '너무나도',
   '이뻐보였다'],
  'pos')]

In [84]:
def term_exists(doc):
#     return {'exists({})'.format(word): (word in set(doc)) for word in selected_words}
    return {word: True for word in doc}

In [85]:
train_xy = [(term_exists(d), c) for d, c in train_reviews] #ㄴ[:1500]
test_xy = [(term_exists(d), c) for d, c in test_reviews] #[:500]

In [86]:
train_xy[:5]

[({'.': True, '더빙': True, '목소리': True, '아': True, '진짜': True, '짜증나네요': True},
  'neg'),
 ({'.': True,
   '가볍지': True,
   '않구나': True,
   '오버연기조차': True,
   '초딩영화줄': True,
   '포스터보고': True,
   '흠': True},
  'pos'),
 ({'너무재밓었다그래서보는것을추천한다': True}, 'neg'),
 ({'.': True,
   '교도소': True,
   '솔직히': True,
   '없다': True,
   '이야기구먼': True,
   '재미는': True,
   '조정': True,
   '평점': True},
  'neg'),
 ({'!': True,
   '너무나도': True,
   '늙어보이기만': True,
   '던스트가': True,
   '돋보였던': True,
   '사이몬페그의': True,
   '스파이더맨에서': True,
   '연기가': True,
   '영화': True,
   '이뻐보였다': True,
   '익살스런': True,
   '커스틴': True,
   '했던': True},
  'pos')]

In [87]:
test_xy[:5]

[({'ㅋ': True, '굳': True}, 'pos'),
 ({'GDNTOPCLASSINTHECLUB': True}, 'neg'),
 ({'.': True,
   '10점': True,
   '나쁘진': True,
   '더더욱': True,
   '뭐야': True,
   '아니잖아': True,
   '않지만': True,
   '이': True,
   '짜리는': True,
   '평점들은': True},
  'neg'),
 ({'.': True,
   '돈주고': True,
   '막장임': True,
   '보기에는': True,
   '않은데': True,
   '완전': True,
   '지루하지는': True},
  'neg'),
 ({'.': True,
   '3D로': True,
   '3D만': True,
   '?': True,
   '개': True,
   '나와서': True,
   '다섯': True,
   '별': True,
   '불편하게': True,
   '심기를': True,
   '아니었어도': True,
   '왜': True,
   '제': True,
   '줬을텐데': True,
   '하죠': True},
  'neg')]

In [88]:
classifier = NaiveBayesClassifier.train(train_xy)

In [89]:
print ('accuracy:', nltk.classify.util.accuracy(classifier, test_xy))

accuracy: 0.83608


In [90]:
classifier.show_most_informative_features()

Most Informative Features
                     1점도 = True              neg : pos    =    206.8 : 1.0
                    재미없어 = True              neg : pos    =    116.5 : 1.0
                      최악 = True              neg : pos    =    108.5 : 1.0
                     최악의 = True              neg : pos    =    106.4 : 1.0
                     0점은 = True              neg : pos    =     85.9 : 1.0
                      망작 = True              neg : pos    =     82.6 : 1.0
                   재미없어요 = True              neg : pos    =     81.3 : 1.0
                    최악이다 = True              neg : pos    =     80.6 : 1.0
                   따뜻해지는 = True              pos : neg    =     77.4 : 1.0
                     아까움 = True              neg : pos    =     74.7 : 1.0
